---
---
# Préparation / consignes
- Votre travail consiste à compléter ce cadre de TP. 
- Vous pouvez (hmmm devez ?) ajouter des blocs de code comme des blocs d'explication.
- Les blocs d'explication sont au format Markdown : 	[markdown](https://www.markdownguide.org/cheat-sheet/).
- Le rendu est une version imprimée de cette ```frame```.
- Pensez à faire des ```commit``` régulièrement
- Il est posible de créer une copie locale de votre travail ```file / Download Notebook```
---
---

 # TP L3 ISIMA : arbres de décision
---
Les principaux points abordés dans ce TP sont :
- La construction des ensembles à manipuler (apprentissage, test, validation)
- La visualisation des données
- Le choix des critères de séparation pour la création d'un arbre de décision
  + cas où la séparation peut s'effectuer sur un unique attribut
  + cas où la sépararation est linéaire, mais doit faire intervenir plusieurs attributs
- La construction de l'arbre
- Le jugement de la qualité de l'apprentissage
- Découverte de sklearn

## Charger les librairies 
- numpy
- pandas
- seaborn
- matplotlib.pyplot

Dans la suite ajouter les imports dans cette cellule.

In [ ]:
# Charger les bibliothèques demandées
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
print("import success")



 ## Charger la base de données 'Iris Species' dans l'environnement
 - Commencer par incorporer la BD dans le kernel : *add data* dans la 'frame' de droite
 - Utiliser la commande pandas permettant de charger le fichier CSV des données
 - N'oubliez pas de vérifier l'apparence du résultat en affichant les premières lignes
 
Rappel : les données se trouvent dans "../input/", dont on peut lister le contenu par :
 ```
import os
print(os.listdir("../input/"))
```
 

In [ ]:
# Charger la BD
#the path
iris_path  = "../input/iris/Iris.csv"

#la lecture
iris = pd.read_csv(iris_path,index_col="Id")

iris





Cette base de donnée est trop bien nettoyée... A des fins pédagogiques, nous allons la déterriorer.
Exécuter la ligne suivante : ```iris[np.random.rand(150,5)<0.05]=np.nan```

In [ ]:
iris[np.random.rand(150,5)<0.05]=np.nan
print("OK!")

## Examiner le contenu de cette base de données
- Lister les 5 premières lignes (préférer```display```  à ```print```)
- Afficher le nombre de lignes ainsi que le nombre de colonnes
- Afficher un résumé statistique simple de cette base

In [ ]:
# Réaliser les premiers affichages
display(iris.head())
a=1
#nbr de ligne et colonne
print(iris.shape)
#resume statistiqu
res_stat=iris.describe()
display(res_stat)


## Vérifier qu'il n'y a pas de données absentes
- Pour chaque attribut, compter le nombre de données manquantes.
- Supprimer les lignes possédant au moins une donnée manquante (c'est la façon la plus simple de se débarrasser du problème)
- Combien d'exemples ont-ils été ainsi perdus ?
- Réfléchir à d'autres façons de traiter les données absentes (conseil : revenir à ce point à la fin du TP)

In [ ]:
# On s'intéresse aux donnés manquantes... 

print(iris.isnull().sum())

index_nan = iris.index[iris.isnull().any(axis=1)]
index_nan
iris.drop(index_nan,0,inplace=True)
display("donnéé supprimé")



In [ ]:
# Comment on aurait pu faire...

# Représentation de la distribution des attributs
## Pour chaque attribut, représenter sa distribution (une courbe par variété)
- Histogramme
- Boîte à moustache
- Diagramme en violon
- Estimation par fonction noyau de la densité

NB : pour obtenir une présentation correcte, il peut être utile d'utiliser ```fig,ax = plt.subplots(paramètres)``` pour définir la présentation des graphiques, puis ```plt.sca(ax[i])``` afin de choisir dans quel sous graphique écrire.

In [ ]:


fig,ax = plt.subplots(nrows=4, ncols=4, figsize=(21, 15))
species = iris.Species.unique()
#boucle pour creer les figures
for i in range(4):
    for j in range(len(species)):
        #creation histrogramme
        sns.distplot(a=iris[iris.Species == species[j]].iloc[:, i], kde=False, ax=ax[0, i])
        #creation boite à moustache
        sns.boxplot(iris[iris.Species == species[j]].iloc[:, i], ax=ax[1, i])
        #creation violin
        sns.violinplot(iris[iris.Species == species[j]].iloc[:, i], ax=ax[2, i])
        #creation desnity
        sns.kdeplot(iris[iris.Species == species[j]].iloc[:, i], ax=ax[3, i])


## Les ensembles de travail
- l'ensemble d'apprentisage : iris_Train, 70% des données
- l'ensemble de validation : iris_Test, 20% des données
- l'ensemble de test : iris_Validation, 10% des données  

### Rappeler la fonction de chacun de ces ensembles
Réponse : iris_train : est l'ensemble de données sur lesquelles 




## Créer ces ensembles
- Charger ```train_test_split``` du module ```sklearn.model_selection```
- Séparer iris en observations (les attributs observables) et classe (la variété)
- Lire le manuel de ```train_test_split```
  - Examiner en particulier l'option stratify, la mettre en oeuvre
  - Se poser la question de l'intérêt de random_state
- Par deux applications de cette fonction, créer les 6 ensembles train_X, train_Y, test_X, test_Y, validation_X, validation_Y (conseil, vérifier les tailles des ensembles obtenus)

In [ ]:


# import puis utilisation pour fabriquer les trois ensembles
from sklearn.model_selection import train_test_split

X = iris.iloc[:,:4]
Y = iris.Species

train_X,rest_X,train_Y, rest_Y= train_test_split(X, Y, train_size= 0.7, test_size= 0.3, stratify=Y)
test_X, validation_X,test_Y, validation_Y= train_test_split(rest_X, rest_Y, train_size=2/3, test_size=1/3, stratify=rest_Y)

display(train_X.shape)
display(train_Y.shape)
display(test_X.shape)
display(test_Y.shape)
display(validation_X)
display(validation_Y)

---
---
# Mise en place manuelle d'un arbre de décision
---
---

# Première découpe : (travail sur **iris_Train**)

## Représenter tous les couples d'attributs possibles
- Diagrammes points ou points
- Densité ou histogramme ou boîte à moustaches

*** (utiliser *pair*grid du module seaborn)***

In [ ]:
# Retour sur des graphiques, mais cette fois pour réaliser l'apprentissage
train_set = pd.concat([train_X, train_Y], axis = 1)
g = sns.PairGrid(train_set, hue='Species')
g = g.map(plt.scatter)
g.add_legend()


## Commenter les graphismes obtenus
- Y a-t-il une variété aisément séparable ?
- Quels attributs permettent de la séparer des deux autres ?
- Quels graphiques ont permis de choisir cet attribut ?

Réponses  :
-oui , petal concernant les lignes et colonnes de petalLength , petalwidth
-l'attributs permettant de separer les deux autres : 
-petallength






## Choisir la racine de l'arbre de décision :
- attribut sur lequel effectuer la séparation 
- valeur du seuil à utiliser

- attribut sur lequel effectuer la séparation 


l'attribut petalLength

- valeur du seuil à utiliser

2.45

## Ecrire une fonction niveau0
- Prenant en entrée une description
- Renvoyant une estimation de la variété d'iris (pour l'instant, il n'y a que deux 'variétés', celle qu'on a séparé et 'le reste'  que l'on note ici ```???```)  

NB : une utilisation de votre fonction peut être par exemple : ```niveau0(iris_Train_X)``` doit renvoyer ```iris_Train_Y``` si l'apprentissage est parfait (ce qui n'est en général pas bon signe...)

NB2 : vous devez renvoyer un DataFrame possédant un unique attribut que vous nommerez

NB3 : mettre en oeuvre apply de pandas


In [ ]:
# Fonction niveau0

def p_classe(r):
    if (r.PetalLengthCm <= 2.45):
        return 'Iris-setosa'
    else:
        return '???'
    
def niveau0(iris_Train_X):
    return iris_Train_X.apply(lambda x: pd.Series(p_classe(x),index=['res']), axis=1)
    
iris_Train_Y = niveau0(train_X)
display(iris_Train_Y.head())



# Niveau suivant de l'arbre
On devrait maintenant construire pour chacune des valeurs de sortie de *niveau0* une fonction permettant d'affiner la classification. Ici, le travail est simplifié, car une des deux classes obtenues par *niveau0* est parfaitement homogène, on ne va donc affiner que la partie corrrespondant à la réponse *???* de la sortie de *niveau0*. Dans le cas général, il faudrait suivre la même procédure sur l'autre sous-arbre.

## Filtrer dans la base de test les éléments dont la réponse par *niveau0* est '???'
- Appeller cette base train_2
- la séparer en train_X_2, train_Y_2

** Si nécessaire, faire un reset d'index : df.reset_index(drop=True, inplace = True) **

In [ ]:
# Ne pas oublier d'effectuer les reset d'index
i = iris_Train_Y[iris_Train_Y['res'] == 'Iris-setosa'].index
train_2 = train_set.drop(i,inplace=False)
train_X_2=train_2.iloc[:, :4]
train_Y_2=train_2[['Species']]
display(train_2.head())
display(train_X_2.head())
display(train_Y_2.head())



## Recommencer le graphique des paires, afin de déterminer la meilleure séparation

In [ ]:
# Reprise des graphiques, en se limittant aux données du sous-arbre '???'

#train_set = pd.concat([train_X, train_Y], axis = 1)
g = sns.PairGrid(train_2, hue='Species')
g = g.map(plt.scatter)
g.add_legend()


## Dur Dur
Il semble ici nettement plus difficile de déterminer la meilleure façon de classer :
- aucune coupe verticale ne semble nettement meilleure que les autres
- aucune coupe diagonale ne semble résoudre le problème
- peut-être existe-il une coupe en dimension supérieure qui serait satisfaisante, mais à partir de la dimension 3, les choses deviennent difficiles à voir...

1) Donner une situation (non présente ici) où il n'y aurait aucune coupe verticale satisfaisante mais où il y aurait une coupe oblique convenable, si possible généraliser à trois variables.

2) Montrer dans un exemple simple en dimension 2 (sur un domaine compact) que par une infinité de coupes verticales il est possible d'obtenir une coupe oblique.

Solution :


## Recherche de la meilleure coupe
On va rechercher parmi tous les attributs celui qui semble permettre la meilleure séparation entre les deux variétés d'iris restantes. Pour cela, on va envisager une coupe selon n'importe quel attribut, et pour n'importe quelle valeur de seuil, puis on réalisera un balayage des coupes verticales possibles, et on conservera la moins mauvaise.

** Il existe des méthodes plus efficaces (heureusement) que celle présentée ici, cf. Séparateurs à Vastes Marges **

Ecrire une fonctionnelle *separe* prenant comme entrées :
- un attribut 'att'
- un seuil
- une étiquette 'A'
- une étiquette 'B'

qui renvoie une fonction qui prend en entrée une situation et qui renvoie 'A' si cette situation a sont attribut *att < seuil* et 'B' sinon

** Normalement vous devriez pouvoir prendre *niveau0* comme base de travail**

In [ ]:

def sp(att,seuil,a,b):
    def parcourir(row):
        if row[att]< seuil:
            return a
        else:
            return b
        

    return parcourir
    
    
    
        
d = train_X_2.apply(lambda x: pd.Series(sp('PetalLengthCm',4.3,'Iris-versicolor','Iris-virginica')(x),index=['res']), axis=1)

        
display(d.head())   

    
    
    

## Choix de la 'meilleure' coupe verticale
Les mesures que vous devez connaître sont  :
- la matrice de confusion  
- Le taux de bonne prédiction (accuracy) $\frac{VP + VN}{VP+VN+FP+FN}$
- Le taux de vrais positifs / rappel (recall, sensitivity) $\frac{VP }{VP+FN}$
- Le taux de vrais négatifs (specificity) $\frac{VN}{VN+FP}$
- La précision $\frac{VP }{VP + FP}$
- La F_1 mesure $2 \times \frac{rappel \times précision}{rappel + précision}$
- La courbe ROC
- Le score ROC  
Si un besoin d'aide sur ces mesures se fait sentir : [Evaluating classifiers](https://www.youtube.com/watch?v=FAr2GmWNbT0)

On choisit de définir la meilleure coupe comme celle ayant le meilleur taux de prédiction.
Ecrire une fonction de balayage renvoyant le tuple *(attribut, seuil, A, B)* (ou la fonction permettant le taux de prédiction) maximisant le taux de prédiction. On nomme mc niveau1_d la fonction de classification obtenue.

NB1 : il n'est pas demandé un algo malin, mais un simple balayage... qui peut prendre un temps important :)

NB2 : penser à incorporermetrics de sklearn...

NB3 : faire attention à bien choisir l'ensemble sur lequel on travaille (apprentissage ?, test ?, validation ?)


## Construction du classifieur chaînant les deux premier classifieurs
Créer une fonction de classification qui enchaîne les deux fonctions **niveau0** et **niveau1_d**, nommer la fonction obtenue **arbre**

---
# Juger de la qualité du travail !!
---
Juger de la qualité du résultat est très important, cela permet
- De choisir entre plusieurs modèles le plus adapté
- De déterminer des pistes d'amélioration d'un modèle
- D'évaluer les capacités du modèle  lorsqu'il sera mis en production  

## Sur quel ensemble doit-on juger de la qualité ?
Expliquer l'intéret de  mesurer la qualité sur chacun des ensembles
- Ensemble d'apprentissage :  
- Ensemble de validation :  
- Ensemble de test :

## Expliquer l'intéret de chacune des mesures précédentes, et proposer un exemple pertinent pour chacune d'elles justifiant son existence
- la matrice de confusion  :
- Le taux de bonne prédiction (accuracy) :
- Le taux de vrais positifs / rappel (recall, sensitivity) :
- Le taux de vrais négatifs (specificity) :
- La précision :
- La F_1 mesure : 

## A l'aide de sklearn.metric, évaluer les différentes mesures, commenter

## Coupes en 'diagonale'
On a choisi d'effectuer des coupes sur un attribut (un côté gauche, et un côté droit). Il est possible également de découper l'espace en deux demi-espaces. Dans l'absolu, outes les découpes sont possibles. On s'intéresse ici des découpes observables par le graphique des paires.  
Réobserver le graphique, et déterminer s'il existe une découpe plus efficace que celle trouvée précédement.
- Il n'est pas demandé de la réaliser (mais vous le pouvez :))
- Exposer une situation à 3 attributs où il n'existe pas de coupe par plan sur 2 paramètres alors qu'il existe un plan séparateur parfait. (ils ne sont linéairement séparables dans aucun des graphiques 'paires' mais sont pourtant linéairement séparables)


### Réponses :

---
# Arbre de décision réalisés par sklearn
---
- importer le module ```tree``` de ```sklearn```
- Etudier la documentation de ```DecisionTreeClassifier```, en particulier la partie **Tips on practical use**
- Construire un classifieur utilisant l'indice de Gini

## Qualification
- Calculer les scores utilisés dans ce TP
- Comparer les scores à ceux obtenus par un classifieur 'bidon' (sklearn.dummy) (à quoi cela sert-il ?)

Rq : il reste un bug dans dummy, si vous obtenez une erreur de type 'no argmax on list', un contournement de ce problème peut être obtenu en reformattant les entrées du classifieur par 'check_X_y'


# Construction du 'meilleur' arbre de décision
- Faire varier les paramètres de construction de l'arbre de décision (bien mettre en pratique les 'Tips')
- Pour chaque série de paramètres, qualifier le résultat sur l'ensemble de test
- Choisir l'arbre le 'meilleur' sur l'ensemble de test  

N'oubliez pas le principe du ** rasoir d'Ockham ** pour effectuer votre choix !!!!!

In [ ]:
# jouer avec les paramètres, et à chaque fois juger de la qualité, jusqu'à obtenir votre 'meilleur arbre


## Au fait, à quoi sert l'ensemble de validation ??
- Utiliser l'ensemble de validation pour donner la qualification finale de votre arbre
- Pourquoi cette qualification ne peut-elle pas être obtenue à partir de l'ensemble de test ?


In [ ]:
# Faire la qualification finale

Réponse à l'utilité de l'ensemble de validation :



## Il est possible de représenter un arbre de décision
- Importer le module graphviz
- utiliser la fonction de ```tree.export_graphviz``` puis ```graphviz.Source``` afin de réaliser une belle représentation graphique

---
# Random Forest
---
- Rappeler le principe des forets d'arbres décisionnelles

- Remplacer le classifieur par arbre de décision par un classifieur par une forêt d'arbres décisionnels

In [ ]:

N=8 
echiquier=[] 
pos_reine=[] 



def init_echiquier():
    for i in range(0,N):
        pos_reine.append(0)
        li=[]
        for k in range(0,N):
            li.append(0)
        echiquier.append(li)


def affiche():
    for element in echiquier :
        print (element)
    print ("__________")     

def poser():
    for i in range(0,N):
        for k in range(0,N):
            echiquier[i][k]=0
    for i in range(0,N): 
        echiquier[i][pos_reine[i]]=1  
    affiche()
    

def valide(li ,co): 
    for i in range(0,co):
        if pos_reine[i]==li or abs(pos_reine[i]-li)==abs(i-co):
            return 0 
    return 1 

def cherche_solution(cur_col): 
    if(cur_col==N): 
        poser()      
    else :
     
	for i in range(0,N): 
            if(valide(i,cur_col)): 
                pos_reine[cur_col]=i  
                cherche_solution(cur_col+1) 

leN=input("Donner Le nombre des reines : ")
N=leN
init_echiquier()
cherche_solution(0) 

# Optimisation des paramètres
- Utiliser une 'gridSearch' de sklearn afin de rechercher les meilleurs paramètres de la forêt

# Choisir les meileurs paramètres en utilisant l'ensemble de  test, expliquer ce qu'est la validation croisée

Réponse :

# Merci d'être allé jusqu'à la fin du TP, j'espère que ce travail vous a aidé à approfondir votre compréhension du cours d'apprentissage artificiel. A bientôt pour la suite :)

In [ ]:
import pandas as pd
Iris = pd.read_csv("../input/iris/Iris.csv")